In [1]:
print('hello')

hello


In [16]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm
import re
from sklearn.feature_extraction.text import CountVectorizer
import time


In [3]:
# Load and preprocess the data
DATASET_COLUMNS = ["sentiment", "ids", "date", "flag", "user", "text"]
DATASET_ENCODING = "ISO-8859-1"
dataset = pd.read_csv('C:/Users/IDEH/Desktop/Sentimental/Data/Sentiment140.csv',
                      encoding=DATASET_ENCODING, names=DATASET_COLUMNS)


In [4]:
# Map sentiment labels to integers (0: negative, 4: positive)
sentiment_map = {0: 0, 4: 1}  # 0: negative, 1: positive
dataset.loc[:, 'sentiment'] = dataset['sentiment'].map(sentiment_map)


In [11]:
# Preprocess the text data
def preprocess(textdata):
    processedText = []
    wordLemm = WordNetLemmatizer()
    urlPattern = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
    userPattern = '@[^\s]+'
    alphaPattern = "[^a-zA-Z0-9]"
    sequencePattern = r"(.)\1\1+"
    seqReplacePattern = r"\1\1"

    for tweet in tqdm(textdata, desc="Processing tweets", unit="tweet"):
        tweet = tweet.lower()
        tweet = re.sub(urlPattern, ' URL', tweet)
        tweet = re.sub(userPattern, ' USER', tweet)
        tweet = re.sub(alphaPattern, " ", tweet)
        tweet = re.sub(sequencePattern, seqReplacePattern, tweet)
        tweetwords = ''
        for word in tweet.split():
            word = wordLemm.lemmatize(word)
            tweetwords += (word + ' ')
        processedText.append(tweetwords)

    return processedText

processed_text = preprocess(dataset['text'].values)

Processing tweets: 100%|████████████████████████████████████████████████| 1600000/1600000 [05:08<00:00, 5194.35tweet/s]


In [14]:
count_vectorizer = CountVectorizer(max_features=20000, stop_words='english', ngram_range=(1, 2), min_df=5, max_df=0.8)

In [17]:
# Vectorize the text data
start_time = time.time()
X_counts = count_vectorizer.fit_transform(tqdm(processed_text, desc="Vectorizing text data", unit="document"))
end_time = time.time()
print(f"Vectorization completed in {end_time - start_time:.2f} seconds")

Vectorizing text data: 100%|████████████████████████████████████████| 1600000/1600000 [01:20<00:00, 19993.52document/s]


Vectorization completed in 114.52 seconds


In [28]:
X_train, X_test, y_train, y_test = train_test_split(X_counts, dataset['sentiment'].values, test_size=0.2, random_state=0)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)


In [29]:
# Convert sparse matrix to tensorflow sparse tensor
def convert_sparse_matrix_to_sparse_tensor(X):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.SparseTensor(indices, coo.data, coo.shape)

In [30]:
X_train_sparse = convert_sparse_matrix_to_sparse_tensor(X_train)
X_val_sparse = convert_sparse_matrix_to_sparse_tensor(X_val)
X_test_sparse = convert_sparse_matrix_to_sparse_tensor(X_test)


In [31]:
# Define the model
def create_model(input_dim):
    model = Sequential([
        Dense(256, activation='relu', input_shape=(input_dim,)),
        Dropout(0.5),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    return model

In [32]:
# Create and compile the model
model = create_model(X_counts.shape[1])
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [33]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


In [34]:
# Train the model
history = model.fit(X_train_sparse, y_train,
                    epochs=10,
                    batch_size=256,
                    validation_split=0.2,
                    callbacks=[early_stopping],
                    verbose=1)


ValueError: `validation_split` is only supported for Tensors or NumPy arrays, found following types in the input: [<class 'tensorflow.python.framework.sparse_tensor.SparseTensor'>]

In [ ]:
#!/usr/bin/env python
# coding: utf-8

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm
import re
from sklearn.feature_extraction.text import CountVectorizer
import time

# Load and preprocess the data
DATASET_COLUMNS = ["sentiment", "ids", "date", "flag", "user", "text"]
DATASET_ENCODING = "ISO-8859-1"
dataset = pd.read_csv('C:/Users/IDEH/Desktop/Sentimental/Data/Sentiment140.csv',
                      encoding=DATASET_ENCODING, names=DATASET_COLUMNS)

# Map sentiment labels to integers (0: negative, 4: positive)
sentiment_map = {0: 0, 4: 1}  # 0: negative, 1: positive
dataset.loc[:, 'sentiment'] = dataset['sentiment'].map(sentiment_map)

# Preprocess the text data
def preprocess(textdata):
    processedText = []
    wordLemm = WordNetLemmatizer()
    urlPattern = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
    userPattern = '@[^\s]+'
    alphaPattern = "[^a-zA-Z0-9]"
    sequencePattern = r"(.)\1\1+"
    seqReplacePattern = r"\1\1"

    for tweet in tqdm(textdata, desc="Processing tweets", unit="tweet"):
        tweet = tweet.lower()
        tweet = re.sub(urlPattern, ' URL', tweet)
        tweet = re.sub(userPattern, ' USER', tweet)
        tweet = re.sub(alphaPattern, " ", tweet)
        tweet = re.sub(sequencePattern, seqReplacePattern, tweet)
        tweetwords = ''
        for word in tweet.split():
            word = wordLemm.lemmatize(word)
            tweetwords += (word + ' ')
        processedText.append(tweetwords)

    return processedText

processed_text = preprocess(dataset['text'].values)

count_vectorizer = CountVectorizer(max_features=20000, stop_words='english', ngram_range=(1, 2), min_df=5, max_df=0.8)

# Vectorize the text data
start_time = time.time()
X_counts = count_vectorizer.fit_transform(tqdm(processed_text, desc="Vectorizing text data", unit="document"))
end_time = time.time()
print(f"Vectorization completed in {end_time - start_time:.2f} seconds")

X_train, X_test, y_train, y_test = train_test_split(X_counts, dataset['sentiment'].values, test_size=0.2, random_state=0)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

Processing tweets:  58%|████████████████████████████▎                    | 926065/1600000 [03:13<02:07, 5266.60tweet/s]

In [35]:


# Convert sparse matrix to tensorflow sparse tensor
def convert_sparse_matrix_to_sparse_tensor(X):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.SparseTensor(indices, coo.data, coo.shape)

X_train_sparse = convert_sparse_matrix_to_sparse_tensor(X_train)
X_val_sparse = convert_sparse_matrix_to_sparse_tensor(X_val)
X_test_sparse = convert_sparse_matrix_to_sparse_tensor(X_test)

# Define the model
def create_model(input_dim):
    model = Sequential([
        Dense(256, activation='relu', input_shape=(input_dim,)),
        Dropout(0.5),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    return model

# Create and compile the model
model = create_model(X_counts.shape[1])
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
history = model.fit(X_train_sparse, y_train,
                    epochs=10,
                    batch_size=256,
                    validation_data=(X_val_sparse, y_val),
                    callbacks=[early_stopping],
                    verbose=1)

# Evaluate the model
y_pred = model.predict(X_test_sparse)
y_pred = (y_pred > 0.5).astype(int)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Vectorizing text data: 100%|████████████████████████████████████████| 1600000/1600000 [01:18<00:00, 20428.73document/s]


Vectorization completed in 112.79 seconds


InvalidArgumentError: {{function_node __wrapped__SerializeManySparse_device_/job:localhost/replica:0/task:0/device:CPU:0}} indices[1] = [0,522] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.

 [Op:SerializeManySparse] name: 